In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl
import numpy as np
import pandas as pd
flowdata = pd.read_excel(r'./flowdata.xlsx', sheet_name='Scenario1')

**Equation**: (w)(KTB) + (x)(KTC) - (y)(SKB) - (z)(SKC) = KTB + KTC - SKB - SKC
- Each set of flow values observed for KTB, KTC, SKB, SKC can be geometrically represented as a line in 4-dimensional space mapped by w, x, y, z
- By iterating through *test_values* for w, x, y, z, we find the point X that minimises the sum of the distance X and all the lines
- Find *num_results* number of points X that provide the *num_results* lowest sum of distances
- Each of w, x, y, z is equal to E/1+E where E is the actual flowmeter error

In [2]:
flowdata

,KTB,SKB,KTC,SKC
0,0,0,390.7,391.7
1,0,0,388.2,388.4
2,0,0,408.9,407.2
3,0,0,413.1,407.5
4,0,0,408.3,400.8
5,0,0,405.5,399.1
6,0,0,394.6,386.4
7,0,0,407.0,399.6
8,0,0,362.8,355.7
9,0,0,408.5,404.9


In [3]:
num_results = 10
results_array = [[[0, 0, 0, 0], sys.float_info.max] for _ in range(num_results)]
results = pd.DataFrame(results_array, columns=['Flowmeter Errors', 'Sum of Distances'])

In [ ]:
test_values = np.arange(-20,21)/10 # -2% to +2% at intervals of 0.1%
flowdata_temp = flowdata
# change the sign of SKB and SKC columns for simpler arithmetic
flowdata_temp['SKB'] *= -1
flowdata_temp['SKC'] *= -1
flowdata_array = flowdata_temp.to_numpy()
for E1 in test_values:
    for E2 in test_values:
        for E3 in test_values:
            for E4 in test_values:
                w = E1/(1+E1)
                x = E2/(1+E2)
                y = E3/(1+E3)
                z = E4/(1+E4)
                sum_of_dist = 0
                for params in flowdata_array:
                    m = sum(params)
                    dist = abs(np.dot(params, [w,x,y,z] + m))/np.linalg.norm(params)
                    sum_of_dist += dist
                if sum_of_dist < results['Sum of Distances'].max():
                    row_to_replace = results['Sum of Distances'].idxmax()
                    results['Flowmeter Errors'].loc[row_to_replace] = [E1,E2,E3,E4]
                    results['Sum of Distances'].loc[row_to_replace] = sum_of_dist

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  if sys.path[0] == '':
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by Inte

In [ ]:
results